In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

### Loading the dataset

In [2]:
training_data = datasets.FashionMNIST(
    root="data/mnist",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data/mnist",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

### Creating the model class

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

### Calling the model

In [5]:
model = NeuralNetwork()

### Hyperparameters

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Loss

In [7]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizer

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### train loop

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        # Forward step
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() # reseting gradients
        loss.backward()
        optimizer.step() # adjust gradients

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss}  [{current} / {size}]")

In [10]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\nAccuracy: {100*correct} %, Avg loss: {test_loss}\n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.3084256649017334  [0 / 60000]
loss: 2.2978627681732178  [6400 / 60000]
loss: 2.2808427810668945  [12800 / 60000]
loss: 2.274210214614868  [19200 / 60000]
loss: 2.250694751739502  [25600 / 60000]
loss: 2.227527379989624  [32000 / 60000]
loss: 2.2348616123199463  [38400 / 60000]
loss: 2.2022626399993896  [44800 / 60000]
loss: 2.1935019493103027  [51200 / 60000]
loss: 2.1652228832244873  [57600 / 60000]
Test Error:
Accuracy: 44.13 %, Avg loss: 2.163416763779464

Epoch 2
-------------------------------
loss: 2.1722309589385986  [0 / 60000]
loss: 2.170527219772339  [6400 / 60000]
loss: 2.11712384223938  [12800 / 60000]
loss: 2.1289358139038086  [19200 / 60000]
loss: 2.079686403274536  [25600 / 60000]
loss: 2.0222082138061523  [32000 / 60000]
loss: 2.0443789958953857  [38400 / 60000]
loss: 1.9697949886322021  [44800 / 60000]
loss: 1.964730143547058  [51200 / 60000]
loss: 1.8932952880859375  [57600 / 60000]
Test Error:
Accuracy: 56.81 %, Avg los